<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Data-Mining/blob/master/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
omidsakaki1370_data_preparation_example_path = kagglehub.dataset_download('omidsakaki1370/data-preparation-example')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# **Laad Data**

In [ ]:
df_example = pd.read_csv('/kaggle/input/data-preparation-example/credit_risk_dataset.csv')
df_backfill = df_example.fillna(method = 'backfill')
missing_values_count = df_backfill.isnull().sum()
print(missing_values_count)
df_backfill.head(10)

# **Normalization**

In [ ]:
fig = plt.figure(figsize=(14, 4))
ax1 = plt.subplot(121)

sns.distplot(df_backfill['person_income']);
ax1.set_title("person_income");

In [ ]:
fig = plt.figure(figsize=(14, 4))
ax1 = plt.subplot(121)

sns.distplot(df_backfill['person_income']);
ax1.set_title("person_income");

df_backfill['person_income'] = np.log(df_backfill['person_income'])

ax1 = plt.subplot(122)
sns.distplot(df_backfill['person_income']);
ax1.set_title("person_income_log");

# **Outliers**

In [ ]:
sns.pairplot(df_backfill, hue='loan_status')
plt.suptitle('Pair Plots by Loan Status', y=1.02)
plt.show()

In [ ]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_out = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_out

df_backfill = remove_outliers(df_backfill, 'person_age')
df_backfill = remove_outliers(df_backfill, 'person_income')
df_backfill = remove_outliers(df_backfill, 'person_emp_length')
df_backfill = remove_outliers(df_backfill, 'cb_person_cred_hist_length')
df_backfill.reset_index(drop=True, inplace=True)

In [ ]:
sns.pairplot(df_backfill, hue='loan_status')
plt.suptitle('Pair Plots by Loan Status', y=1.02)
plt.show()

# **Encoder**

In [ ]:
df_backfill.info()

In [ ]:
df_encoder_example = df_backfill.copy()

## **LabelEncoder**

In [ ]:
encoder=LabelEncoder()

df_encoder_example['person_home_ownership']=encoder.fit_transform(df_encoder_example['person_home_ownership'])
df_encoder_example['loan_intent']=encoder.fit_transform(df_encoder_example['loan_intent'])
df_encoder_example['loan_grade']=encoder.fit_transform(df_encoder_example['loan_grade'])
df_encoder_example['cb_person_default_on_file']=encoder.fit_transform(df_encoder_example['cb_person_default_on_file'])

In [ ]:
df_encoder_example.head(10)

In [ ]:
df_encoder_example.info()

## **One Hot**

In [ ]:
df_backfill.head(10)

In [ ]:
edt = pd.get_dummies(df_backfill['loan_grade'], dtype=int)

edt

In [ ]:
df_encoder_example2 = df_backfill.join(edt)
df_encoder_example2.drop(['loan_grade'], axis=1, inplace=True)

df_encoder_example2.head(10)

# **Scaler**

In [ ]:
df_encoder_example.describe()

## **StandardScaler**

In [ ]:
standard_scaler = StandardScaler()

data_scaled = standard_scaler.fit_transform(df_encoder_example)

fig = plt.figure(figsize=(10, 4))

sns.distplot(data_scaled);

## **MinMaxScaler**

In [ ]:
minmax_scaler = MinMaxScaler()

data_scaled = minmax_scaler.fit_transform(df_encoder_example)

fig = plt.figure(figsize=(10, 4))

sns.distplot(data_scaled);